In [2]:
!pip install feedparser # 설치 : RSS에서 xml태그별 정보를 추출(예: title, link, ...)
!pip install newspaper3k # 설치 : 인터넷신문기사분석(Article()을 사용하기 위한 목적) 파이썬 3버전이라 3k, 링크를 기사 들어가서 분석
!pip install konlpy # 설치 : 한국어 형태소 분석기 (명사만 추출할 목적)
!pip install soynlp # 설치 : 한국어 형태소 분석기 (명사만 추출할 목적)
!pip install lxml[html_clean] # 신규추가(2024년 가을)
import feedparser # !pip install feedparser
from newspaper import Article # !pip install newspaper3k
from konlpy.tag import Okt # !pip install konlpy
from collections import Counter # 본문에 몇번이나 명사가 나오는지 확인(TF구현용)
from bs4 import BeautifulSoup # 글에 존재할지 모르는 단어 추출

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.2 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=a411682a5fcf13f7308e0df622b2e9b2237028c0705d7aad44e9146a88bd1bae
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.9 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13539 sha256=09e446def953be02ce1a71c8b1943d13f3675c11b30e41ece3db13602102219c
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df4

In [3]:
# [단계1] 모든 RSS파일(xml형식)을 돌아다니면서 기사의 제목/link를 추출
# urls는 우리가 검색할 RSS파일들의 목록을 list로 만든 것
urls = ["https://rss.etnews.com/Section901.xml",
        "https://rss.etnews.com/Section902.xml",
        "https://rss.etnews.com/Section903.xml",
        "https://rss.etnews.com/Section904.xml"]
# 아래의 함수는 RSS목록의 list인 urls를 받아서 그 list에 존재하는
# 모든 RSS의 안에 있는 모든 기사들의 title과 link를 추출함
def crawl_rss(urls):
  array_rss = [] # 함수시작하는 시점에 빈 리스트를 만듦. 여기에 모든 기사의 title, link를 넣을 것임
  titles_rss = set() # [중복기사제거] 기사제목들의 집합을 구성함(집합은 중복을 허용하지 않기 때문임)
  for index_url in urls: # urls 리스트안에 하나씩 xml을 방문 (4번 방문예정 : 901, 902, 903, 904)
    print("[Crawl RSS]", index_url) # 현재 어떤 xml파일을 방문중인지 표시
    parse_rss = feedparser.parse(index_url) # 현재 xml파일을 파싱 후, 결과를 parse_rss에 저장
    for p in parse_rss.entries: # parse_rss에 있는 모든 entries/기사를 검색하면서
      if p.title not in titles_rss : # [중복기사제거] 기사제목집합에 현재 title이 업슬 때만 추가
        array_rss.append({'title':p.title,'link':p.link}) # append함수로 array_rss에 title과 link를 붙여넣음
        titles_rss.add(p.title) # [중복기사제거] 집합에 기사제목을 추가
      else:
        print("Duplicated Article:", p.title) # [중복기사제거] 중복되는 기사의 제목을 출력
  return array_rss
list_articles = crawl_rss(urls)
print(list_articles)

[Crawl RSS] https://rss.etnews.com/Section901.xml
[Crawl RSS] https://rss.etnews.com/Section902.xml
Duplicated Article: 엔비디아, 애플 꺾고 전세계 시총 1위 등극
Duplicated Article: 기아, 인증 중고차 토탈케어 서비스 '리멤버스' 출시
[Crawl RSS] https://rss.etnews.com/Section903.xml
Duplicated Article: LG엔솔, 배터리SW 사업화…현대차와 사용료 협상
Duplicated Article: 외산 장비 설 곳 잃어가는 中 반도체 시장
Duplicated Article: 韓·美, 원자력수출협력 합의…산업부 “제2 웨스팅하우스 분쟁 막는다”
Duplicated Article: 서초 서리풀 그린벨트 푼다…고양·의왕·의정부 신규택지 5만호 공급
[Crawl RSS] https://rss.etnews.com/Section904.xml
Duplicated Article: 엔비디아, 애플 꺾고 전세계 시총 1위 등극
Duplicated Article: 기아, 인증 중고차 토탈케어 서비스 '리멤버스' 출시
Duplicated Article: 위고페어, '원클릭 위조상품 신고서비스' 출시
Duplicated Article: 쿠팡, 3분기 매출 10조6900억 '역대 최대'...고객도 11% 증가
Duplicated Article: '장 담그기' 유네스코 인류무형유산으로… “밥·김치 등 韓 문화의 핵심”
Duplicated Article: 막오른 `미국의 선택`...해리스-트럼프 초접전
Duplicated Article: 日 “공주는 안돼”… 유엔 '여성 왕위 계승' 권고 사실상 거부
Duplicated Article: 축구장에 날벼락…페루 선수 8명 쓰러졌다
Duplicated Article: 美 뉴욕서 6명 목숨 앗아간 '지하철 서핑'… 과연 뭐길래?
Duplicated Article: 짐 켈러·조주완 CEO 회

In [4]:
# [단계2] list에 존재하는 모든 기사드릐 link를 하나씩 돌아다니며 본문 title과 text 추출
# 아래의 함수는 하나의 url을 입력받아서, 그 링크를 타고 들어가, 그 안에 title과 text를 추출함.
def crawl_article(url, language='ko'):
  print("[Crawl Article]", url) # 현재 text를 추출할 기사의 url 출력
  article = Article(url, language=language) # Article을 사용하여 그 url을 입력하고, 언어옵션지정
  article.download() # 해당하는 url기사 다운로드
  article.parse() # 해당하는 url기사 분석
  return article.title, article.text # 해당하는 url기사 title과 text 출력

for article in list_articles: # 기존에 만든 list_articles에서 하나하나의 기사를 방문하면서
  _, text = crawl_article(article['link']) # 그 기사의 link를 crawl_article함수에 넣어 본문 추출
  # _, text = crawl_article(article['link'] , 'en') # 영어신문기사일 겨우의 옵션 적용
  article['text'] = text # 그 추출한 본문을 list_articles에 'text'라는 속성으로 새로 만들어 저장
print(list_articles[0])

[Crawl Article] https://www.etnews.com/20241106000042
[Crawl Article] https://www.etnews.com/20241106000041
[Crawl Article] https://www.etnews.com/20241105000329
[Crawl Article] https://www.etnews.com/20241106000001
[Crawl Article] https://www.etnews.com/20241105000378
[Crawl Article] https://www.etnews.com/20241105000263
[Crawl Article] https://www.etnews.com/20241105000379
[Crawl Article] https://www.etnews.com/20241105000371
[Crawl Article] https://www.etnews.com/20241105000253
[Crawl Article] https://www.etnews.com/20241105000404
[Crawl Article] https://www.etnews.com/20241105000115
[Crawl Article] https://www.etnews.com/20241105000374
[Crawl Article] https://www.etnews.com/20241105000310
[Crawl Article] https://www.etnews.com/20241105000240
[Crawl Article] https://www.etnews.com/20241104000451
[Crawl Article] https://www.etnews.com/20241105000350
[Crawl Article] https://www.etnews.com/20241105000334
[Crawl Article] https://www.etnews.com/20241105000333
[Crawl Article] https://www.

In [5]:
# [단계3] 긁어온 본문 text에서 명사추출(키워드, 빈도수)
def get_keywords(text, nKeywords=10): # 키워드추출함수(빈도수고려: TF), 디폴트로 10개
  list_keywords = [] # 비어있는 키워드리스트를 먼저 만든다. 추후에 append를 써서 모든 기사에 대한 내용 추가
  spliter = Okt() # konlpy에 의해서 문장을 형태소별로 쪼개는 기능을 위해 spliter 생성
  nouns = spliter.nouns(text) # 입력받은 text에서 명사만 추출
  count = Counter(nouns) # 추출된 명사들의 출현빈도를 카운트
  for n, c in count.most_common(nKeywords): # 높은건 위, 낮은건 아래로 10번 연산
    item = {'keyword':n,'count':c} # {'keyword', 'count'}의 형식으로 저장
    list_keywords.append(item) # 위의 포멧으로 list_keywords에 저장
  return list_keywords


for article in list_articles: # 기존에 만든 list_articles에서 하나하나의 기사를 방문하면서
  keywords = get_keywords(article['text']) # 그 기사의 text를 get_keywords함수에 넣어 키워드/빈도 추출
  article['keywords'] = keywords # 그 추출한 키워드와 빈도수를 list_articles에 저장
print(list_articles[:5])

[{'title': '엔비디아, 애플 꺾고 전세계 시총 1위 등극', 'link': 'https://www.etnews.com/20241106000042', 'text': "엔비디아가 5일(현지 시각) 애플을 제치고 세계에서 가장 큰 시가총액의 상장 기업이 됐다. 장중 기록한 것은 앞서 몇 차례 있었지만 종가 기준으로 시총 1위에 오른 것은 이번이 처음이다.\n\n\n\n5일 나스닥 증시에서 엔비디아는 전 거래일 대비 2.84%(+3.86달러) 상승한 139.91달러에 마감했다. 시가총액은 3조 4320억 달러로 전 세계 시총 1위다.\n\n\n\n2위로 밀려난 애플도 0.65%(+1.44달러) 오른 223.45달러로 마감하며 시총 3조 3776억달러를 기록했다.\n\n\n\n애플 역시 올해 들어 주가가 17% 상승하며 견조한 기록을 냈지만, 엔비디아는 같은 기간 거의 주가가 3배 가까이 뛰어오르며 마이크로소프트(MS), 알파벳, 아마존닷컴은 물론 애플 시총을 뛰어넘어 1위에 올랐다.\n\n\n\n엔비디아가 처음 애플 시총을 뛰어넘은 것은 올해 6월이다. 3위에 머물던 엔비디아는 장중 MS와 애플을 뛰어넘고 1위에 올랐으나 '인공지능(AI) 거품론'으로 올 여름 상승분 일부를 반환했다.\n\n\n\n한편, 엔비디아는 지난 1일 인텔을 대체해 S&P 다우존스지수에 편입됐다. 다우지수는 미국 주요 업종을 대표하는 우량주 30개 종목으로 구성된다.\n\n\n\n서희원 기자 shw@etnews.com", 'keywords': [{'keyword': '엔비디아', 'count': 6}, {'keyword': '애플', 'count': 6}, {'keyword': '총', 'count': 5}, {'keyword': '위', 'count': 5}, {'keyword': '기록', 'count': 3}, {'keyword': '것', 'count': 3}, {'keyword': '상승', 'count': 3}, {'keyword': '세계', 'count': 2}, {'k

In [18]:
# [단계4] 검색어를 입력받아서 그 검색어를 가지고 있는 문서를 출력
query = input("질의어를 입력하세요 : ") # 쿼리를 입력 받음

# 아래의 함수는 쿼리를 입력받은 후에, 정해진 문서의 keywords의 리스트에서
# 쿼리가 그 keyword 중의 하나로 존재하는지 검색
def search_articles(query, list_keywords): # 쿼리가 키워드리스트에 있으면 빈도수 출력(없으면 0)
  nWords = 0 # 아래의 if문에 걸리지 않으면(즉, 쿼리가 키워드에 없으면) 0을 출력하기 위한 초기값
  for kw in list_keywords: # 키워드 리스트를 하나씩 kw로 검색
    if query == kw['keyword']: # 만약에 쿼리와 동일한 키워드가 존재한다면
      nWords = kw['count'] # 그렇다면, 그 키워드에 해당하는 count가 nWords가 됨
  return nWords # 결과적으로 쿼리의 출현횟수가 출력됨


sorted_query = [] # 정렬된 기사 저장

for article in list_articles: # 기존에 만든 list_articles에서 하나의 기사를 방문하면서
  TF = search_articles(query, article['keywords']) # search_articles함수로 쿼리의 빈도수 추출
  if TF != 0: # 만약에 쿼리의 빈도수가 0이 아니면, 쿼리를 키워드로 가지고 있는 문서이므로, 관련 정보 출력
    sorted_query.append([TF, article['title'],article['link']]) # TF와 article을 튜플로 저장

# (아래) 리스트를 정렬하는데,
# 역순으로 TF값이 큰것부터 나오게하고,
# key값을 x[0]으로하여 리스트속서으이 가장 앞에 있는 TF를 기준으로 정렬
sorted_query.sort(reverse=True, key=lambda x:x[0])
print('-----------------------')
for article in sorted_query:
    print('[TF]',article[0])
    print(article[1])
    print(article[2])
    print('-----------------------')

질의어를 입력하세요 : 중국
-----------------------
[TF] 22
외산 장비 설 곳 잃어가는 中 반도체 시장
https://www.etnews.com/20241105000240
-----------------------
[TF] 8
세계 2위 폭스바겐 위기…3위 현대차 추격에 '지각변동' 예고
https://www.etnews.com/20241101000194
-----------------------
[TF] 7
중국, EU '중국산 전기차 고율 관세' WTO에 추가 제소
https://www.etnews.com/20241105000303
-----------------------
